# Linear Programming with Gurobi in Python

Shows how to solve using python and Gurobipy the following standar linear programming problems 

$$
\begin{align}
\underset{x}{\min} \quad & c^Tx \\
\text{s.t.} \quad & Ax \leq b\\
 & l \leq x \leq u\
\end{align}
$$

where $x\in\mathbb{R}^n$, $c\in\mathbb{R}^n$, $l\in\mathbb{R}^n$, $u\in\mathbb{R}^n$, $A\in\mathbb{R}^{m\times n}$, $b\in\mathbb{R}^m$. Elements of $A,b,c,l,u$ are randomly generating using normal or uniform distributions.

## Requirements
IMPORTANT NOTE: Run this before the rest of the cells.

In [2]:
!pip install -q gurobipy
import time
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Create random vectors and matrix of linear problem

In [3]:
# Number of variables (nvar) and constraints (ncon)
nvar = 1000
ncon = 200
# Random vectors and matrix of linear problem
c = np.random.normal(0,1,size=nvar)
A = np.random.normal(0,5,size=(ncon,nvar))
b = np.random.uniform(0,30,ncon)
u = np.random.uniform(10,20,nvar)
l = np.random.uniform(0,10,nvar)

## GUROBIPY ([link](https://pypi.org/project/gurobipy/))

### Matrix notation

In [4]:
 start = time.time()
 # Model
 m = gp.Model()
 # Variables
 x = m.addMVar(shape=nvar,lb=l,ub=u,vtype=GRB.CONTINUOUS)
 # Objective function
 m.setObjective(c @ x, GRB.MINIMIZE)
 # Constraints
 m.addConstr(A @ x <= b)
 # Solve the problem
 m.optimize()
 # Print results
 print('obj =',m.ObjVal)
 print('time =',time.time()-start)


Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 200 rows, 1000 columns and 200000 nonzeros
Model fingerprint: 0x7ac76de4
Coefficient statistics:
  Matrix range     [3e-06, 2e+01]
  Objective range  [6e-05, 4e+00]
  Bounds range     [3e-04, 2e+01]
  RHS range        [9e-02, 3e+01]
Presolve time: 0.06s
Presolved: 200 rows, 1000 columns, 200000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.4150110e+06   3.807878e+07   0.000000e+00      0s
     369   -1.9070601e+03   0.000000e+00   0.000000e+00      0s

Solved in 369 iterations and 0.13 seconds (0.17 work units)
Optimal objective -1.907060128e+03
obj = -1907.0601275292913
time = 0.18552279472351074


**bold text**### Constraint notation

In [ ]:
start = time.time()
# Model
m = gp.Model()
# Variables
x = m.addVars(nvar,lb=l,ub=u,vtype=GRB.CONTINUOUS)
# Objective function
m.setObjective(gp.LinExpr(c,x.values()),GRB.MINIMIZE)
# Constraints
m.addConstrs((gp.LinExpr(A[j,:],x.values()) <= b[j] for j in range(ncon)))
# Solve the problem
m.optimize()
# Print results
print('obj = ',m.ObjVal)
print('time =',time.time()-start)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 200 rows, 1000 columns and 200000 nonzeros
Model fingerprint: 0x180fd1dc
Coefficient statistics:
  Matrix range     [8e-05, 2e+01]
  Objective range  [4e-04, 4e+00]
  Bounds range     [3e-03, 2e+01]
  RHS range        [5e-02, 3e+01]
Presolve time: 0.10s
Presolved: 200 rows, 1000 columns, 200000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.4296535e+06   7.927222e+06   0.000000e+00      0s
     549   -1.9319535e+03   0.000000e+00   0.000000e+00      0s

Solved in 549 iterations and 0.26 seconds
Optimal objective -1.931953488e+03
obj =  -1931.9534884057082
time = 0.48479795455932617


# **Modelado de batería con Gurobi**

Ejemplo real de modelo de un problema de optimización con Gurobipy. Se programa el modelo de una batería. Está basado en el paper https://arxiv.org/abs/2204.08240

In [12]:
start = time.time()  # Inicia cronómetro

# Constants
E_min = 0.7
E_max = 2
E_0 = 1.5
P_max_c = 0.8
P_max_d = 1
eta_c = 0.85
eta_d = 0.9

p_sig = - 0.5  # Señal de batería. Positivo es descarga

# Model
m = gp.Model()

# Variables
e = m.addVar(vtype=GRB.CONTINUOUS, lb=E_min, ub=E_max, name='e')
p_c = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name='p_c')
p_d = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name='p_d')
z = m.addVar(vtype=GRB.BINARY, name='z')

# Objective function
m.setObjective(((p_d - p_c) - p_sig) ** 2, GRB.MINIMIZE)

# Constraints
m.addConstr(e - eta_c * p_c + 1 / eta_d * p_d == E_0)
m.addConstr(p_d <= P_max_d * z)
m.addConstr(p_c <= P_max_c * (1 -z))

# Solve the problem
m.optimize()

# Print results
print('time =',time.time()-start)
print('obj = ',m.ObjVal)
for v in m.getVars():
  print(str(v.VarName) + "=" + str(round(v.x, 2)))



Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 3 rows, 4 columns and 7 nonzeros
Model fingerprint: 0xd98144a4
Model has 3 quadratic objective terms
Variable types: 3 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 4e+00]
  Bounds range     [7e-01, 2e+00]
  RHS range        [8e-01, 2e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 4 nonzeros
Presolved model has 3 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Found heuristic solution: objective 0.0000000

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0   